In [1]:
### This notebook compared the magic of Scrooge ensembles wrt the magic of eigenstates

import qutip as qt
from qutip import *

##Qiskit libraries
import qiskit
from qiskit.quantum_info import Statevector
from qiskit.quantum_info import partial_trace
from qiskit.quantum_info import entropy

from qiskit import QuantumCircuit
import qiskit_aer 
from qiskit.quantum_info import state_fidelity
from qiskit_aer import AerSimulator
from qiskit import transpile
from qiskit.circuit.library import Initialize
from qiskit.visualization import plot_bloch_multivector
import numpy as np
import matplotlib.pyplot as plt
import sys
from qiskit_aer.primitives import SamplerV2 as Sampler
from qiskit_aer.primitives import Estimator
from qiskit.quantum_info import SparsePauliOp
from qiskit.quantum_info import Operator

#scipy
import scipy
import itertools
from scipy.special import psi, polygamma
import math

from scipy.linalg import sqrtm
from scipy.linalg import expm
from scipy.optimize import root_scalar

##Libraries in Toby's Code

import time
import operator
from functools import reduce

In [2]:
def genFockOp(op,position,size,levels=2,opdim=0):
    opList=[qt.qeye(levels) for x in range(size-opdim)]
    opList[position]=op
    return qt.tensor(opList)


def numberToBase(n, b,n_qubits):
    if n == 0:
        return np.zeros(n_qubits,dtype=int)
    digits = np.zeros(n_qubits,dtype=int)
    counter=0
    while n:
        digits[counter]=int(n % b)
        n //= b
        counter+=1
    return digits[::-1]


    
def get_all_paulis_N(n_qubits):
    ##get all pauli operators
    n_paulis=4**n_qubits

    pauli_list=np.zeros([n_paulis,n_qubits],dtype=int)
    for k in range(n_paulis):
        pauli_list[k,:]=numberToBase(k,4,n_qubits)

    levels=2
    opZ=[genFockOp(qt.sigmaz(),i,n_qubits,levels) for i in range(n_qubits)]
    opX=[genFockOp(qt.sigmax(),i,n_qubits,levels) for i in range(n_qubits)]
    opY=[genFockOp(qt.sigmay(),i,n_qubits,levels) for i in range(n_qubits)]

    
    opId=genFockOp(qt.qeye(levels),0,n_qubits)

    pauli_op_list=[]
    for k in range(n_paulis):
        pauli_string=pauli_list[k]
        pauli=opId
        for i in range(n_qubits):
            if(pauli_string[i]==1):
                pauli=opX[i]*pauli
            elif(pauli_string[i]==2):
                pauli=opY[i]*pauli
            elif(pauli_string[i]==3):
                pauli=opZ[i]*pauli
                
        pauli_op_list.append(pauli)
    return pauli_op_list,pauli_list





def get_conversion_matrix_mod_add_index(base_states):
    ##precompuation for computing renyi stabilizer entropy
    n_qubits=len(base_states[0])
    mag = len(base_states)
    to_index=2**np.arange(n_qubits)[::-1]
    conversion_matrix=np.zeros([mag,mag],dtype=int)
    for j_count in range(mag):
        base_j=base_states[j_count]
        k_plus_j=np.mod(base_states+base_j,2)
        k_plus_j_index=np.sum(k_plus_j*to_index,axis=1)
        conversion_matrix[j_count,:]=k_plus_j_index

    return conversion_matrix

def get_conversion_matrix_binary_prod(base_states):
    ##precompuation for computing renyi stabilizer entropy
    mag = len(base_states)
    conversion_matrix=np.zeros([mag,mag],dtype=int)
    
        
    for i_count in range(mag):
        base_i=base_states[i_count]
        binary_product=np.mod(np.dot(base_states,base_i),2)
        conversion_matrix[i_count,:]=(-1)**binary_product
        

    return conversion_matrix

def pauli_spectrum_fast2(state,conversion_matrix_mod_add_index,conversion_matrix_binary_prod,alpha=[2]):
    ##fast code to compute renyi entropy which uses numpy to full extent
    #requires precomputation with get_conversion_matrix_binary_prod and get_conversion_matrix_mod_add_index
    coeffs=state  #.data.toarray()[:,0]
    n_qubits = int(np.log2(len(coeffs)))
    
    prob_list_full=np.abs(np.dot(np.conjugate(coeffs)*conversion_matrix_binary_prod, coeffs[conversion_matrix_mod_add_index] ))**2
    
    #print(prob_list_full)
    epsilon_cutoff=10**-30
    prob_list=2**(-n_qubits)*prob_list_full[prob_list_full>epsilon_cutoff]

    return prob_list

def get_magic_exact(state,conversion_matrix_mod_add_index,conversion_matrix_binary_prod,alpha=[2]):
    """
    Computes magic of state using fast pauli spectrum method
    """
    
    prob_list=pauli_spectrum_fast2(state,conversion_matrix_mod_add_index,conversion_matrix_binary_prod,alpha=[alpha])
    renyi_fast_list=[]

    n_qubits = int(np.log2(len(state)))
    d = 2**n_qubits


    for alpha_p in alpha:
        if(alpha_p==1):
            renyi_fast= (-np.sum(prob_list*np.log2(prob_list)) - n_qubits)*(d/(d-1))  #-n_qubits*np.log(2)


    
        else:
        

            renyi_fast = 1/(1-alpha_p)*(np.log2(np.sum(prob_list**(alpha_p)) - 1/d**alpha_p) + alpha_p*np.log2(d) - np.log2(d-1) ) #-n_qubits*np.log(2)

        renyi_fast_list.append(renyi_fast)
    
    
    
    return np.array(renyi_fast_list)

In [3]:
#
def Xp(i,N):  #i goes from 1 to N
    str1 = ''
    
    for k in range(i-1):
        str1 += 'I'
    str1 += 'X'
    for k in range(N-i):
        str1 += 'I'
    return SparsePauliOp(str1)

def Yp(i,N):  #i goes from 1 to N
    str1 = ''
    
    for k in range(i-1):
        str1 += 'I'
    str1 += 'Y'
    for k in range(N-i):
        str1 += 'I'
    return SparsePauliOp(str1)

def Zp(i,N):  #i goes from 1 to N
    str1 = ''
    
    for k in range(i-1):
        str1 += 'I'
    str1 += 'Z'
    for k in range(N-i):
        str1 += 'I'
    return SparsePauliOp(str1)

def ZZp(i,N): #i goes from 1 to N-1
    str1 = ''
    
    for k in range(i-1):
        str1 += 'I'
    str1 += 'ZZ'
    for k in range(N-i-1):
        str1 += 'I'
    return SparsePauliOp(str1)

def YZp(i,N): #i goes from 1 to N-1
    str1 = ''
    
    for k in range(i-1):
        str1 += 'I'
    str1 += 'YZ'
    for k in range(N-i-1):
        str1 += 'I'
    return SparsePauliOp(str1)

def hlist(N,J,h):
    h_list = [J]
    for i in range(1,N-1):
        h_list.append(h)
    h_list.append(-J)
    return h_list

def H_tfim(N,g,J,h_list):
    op = 0
    for i in range(1,N):
        op += g*Xp(i,N) + h_list[i-1]*Zp(i,N) + J*ZZp(i,N)
    op+=g*Xp(N,N) + h_list[N-1]*Zp(N,N)
    return op

N_list = [10,11,12,13]

"""for N in N_list:
    print(f'Calculating for N={N}')
    
    # Define the Hamiltonian for the TFIM
    # N is the number of spins
    # g is the transverse field strength
    # J is the coupling strength
    # h_list is a list of local fields on each spin

    d = 2**N
    J = 1
    g = 1.1 #(np.sqrt(5) + 5)/8
    h = 0.35
    h_list = hlist(N,0.25,h)

    H = H_tfim(N,g,J,h_list)


    start = time.time()
    eigvals, eigvecs = np.linalg.eigh(H)
    end = time.time()
    print(f'Time taken for N={N}: {end - start} seconds')
    np.savetxt(f'Test_data/mfim_eigvecs_h={h},g={g},N={N}.txt',np.real(eigvecs))
    np.savetxt(f'Test_data/mfim_eigvals_h={h},g={g},N={N}.txt',np.real(eigvals))"""

"for N in N_list:\n    print(f'Calculating for N={N}')\n    \n    # Define the Hamiltonian for the TFIM\n    # N is the number of spins\n    # g is the transverse field strength\n    # J is the coupling strength\n    # h_list is a list of local fields on each spin\n\n    d = 2**N\n    J = 1\n    g = 1.1 #(np.sqrt(5) + 5)/8\n    h = 0.35\n    h_list = hlist(N,0.25,h)\n\n    H = H_tfim(N,g,J,h_list)\n\n\n    start = time.time()\n    eigvals, eigvecs = np.linalg.eigh(H)\n    end = time.time()\n    print(f'Time taken for N={N}: {end - start} seconds')\n    np.savetxt(f'Test_data/mfim_eigvecs_h={h},g={g},N={N}.txt',np.real(eigvecs))\n    np.savetxt(f'Test_data/mfim_eigvals_h={h},g={g},N={N}.txt',np.real(eigvals))"

In [4]:
##We set a fixed beta, and find the mid-spectrum eigenvalue closest to that beta

def HaarState(N):
    coeffs = []
    for i in range(2**N):
        z = np.random.randn() + 1j * np.random.randn()
        coeffs.append(z)
    coeffs = np.array(coeffs)
    hst = Statevector(coeffs)/np.linalg.norm(coeffs)
    return hst

def scrooge_ensemble(beta,N,H,M=100):
    rho = expm(-beta * SparsePauliOp.to_matrix(H))
    rho = rho / np.trace(rho)#Numpy array

    #convert to qiskit operators
    
    sqrt_rho = sqrtm(rho)
    rho_op = Operator(rho)

    #Form M random Haar states

    rand_states = []
    rand_states2 = []
    for i in range(M):
        rand_states.append(HaarState(N).data)
        rand_states2.append(HaarState(N))

    #Forming the Scrooge ensemble
    fin_states = [(np.dot(sqrt_rho,rand_state))/np.linalg.norm(np.dot(sqrt_rho,rand_state)) for rand_state in rand_states]
    #fin_states = [state.data[0] for state in fin_states]  # Convert to numpy arrays
    prob_list = [np.real(rand_state.expectation_value(rho_op)) for rand_state in rand_states2]

    prob_list_normalised = [p/np.sum(prob_list) for p in prob_list]
    return fin_states, prob_list_normalised

def beta_to_eigval(beta, eigvals,H):
    expH = expm(-beta * H)
    rho = expH / np.trace(expH)

    val = np.trace(H @ rho).real
    eigval_index = np.argmin(np.abs(eigvals - val))
    return eigval_index

def get_magic_eigvals(h,g,J,N,beta,width=0,alpha=[2]):
    eigvals = np.loadtxt(f'Test_data/mfim_eigvals_h={h},g={g},N={N}.txt')
    eigvecs = np.loadtxt(f'Test_data/mfim_eigvecs_h={h},g={g},N={N}.txt')
    H = H_tfim(N,g,J,hlist(N,0.25,h))

    start=time.time()

    base_states=np.array([numberToBase(i, 2,N) for i in range(2**N)],dtype=int)

    ##these steps need only be run once
    conversion_matrix_binary_prod=get_conversion_matrix_binary_prod(base_states)
    conversion_matrix_mod_add_index=get_conversion_matrix_mod_add_index(base_states)



    
    eigval_index = beta_to_eigval(beta, eigvals, SparsePauliOp.to_matrix(H))
    a = eigval_index - width
    b = eigval_index + width
    magic_list = np.zeros(len(alpha))
    for i in range(a,b+1):
        eigvec = eigvecs[i]
        c = get_magic_exact(eigvec, conversion_matrix_mod_add_index, conversion_matrix_binary_prod,alpha=alpha)
        magic_list += c
    magic_list/= 2*width + 1  #Uniform average over the eiegnvalues in the range [a,b]
    return magic_list




In [5]:
##Now, we compute the magic of eigenstates and the magic of the Scrooge ensemble

beta = 0 #Fixed beta
M=10 #Number of states in the Scrooge ensemble
alpha = [0.5,1,2,3] #Renyi entropies to compute
width = 0 #Width of the range of eigenvalues to consider around the mid-spectrum eigenvalue

data_eigvals = np.zeros((len(alpha), len(N_list)))
data_scrooge = np.zeros((len(alpha), len(N_list)))
for col in range(len(N_list)):
    N = N_list[col]
    print(f'Calculating for N={N}')
    h = 0.35
    g = 1.1 #(np.sqrt(5) + 5)/8
    J = 1

    # Get magic of eigenstates
    magic_eigvals = get_magic_eigvals(h, g, J, N, beta, width=width, alpha=alpha)
    
    # Get magic of Scrooge ensemble
    fin_states, prob_list_normalised = scrooge_ensemble(beta, N, H_tfim(N,g,J,hlist(N,0.25,h)), M=M)
    start=time.time()

    base_states=np.array([numberToBase(i, 2,N) for i in range(2**N)],dtype=int)

    ##these steps need only be run once

    conversion_matrix_binary_prod=get_conversion_matrix_binary_prod(base_states)
    conversion_matrix_mod_add_index=get_conversion_matrix_mod_add_index(base_states)
 

    
    magic_scrooge = np.zeros(len(alpha))
    for j in range(M):
        magic_scrooge += get_magic_exact(fin_states[j], conversion_matrix_mod_add_index, conversion_matrix_binary_prod, alpha=alpha) * prob_list_normalised[j]
    
    print(f'Magic of eigenstates for N={N}: {magic_eigvals}')
    print(f'Magic of Scrooge ensemble for N={N}: {magic_scrooge}')

    data_eigvals[:, col] = magic_eigvals
    data_scrooge[:, col] = magic_scrooge
    print()

# Save the data to files
np.savetxt(f'Test_data/magic_eigvals_beta={beta}.txt', data_eigvals)
np.savetxt(f'Test_data/magic_scrooge_beta={beta}.txt', data_scrooge)

Calculating for N=10
Magic of eigenstates for N=10: [8.14072865 7.56333385 6.72934982 5.55385114]
Magic of Scrooge ensemble for N=10: [9.35058296 8.95020193 8.41939488 8.05252692]

Calculating for N=11
Magic of eigenstates for N=11: [9.17483951 8.62786092 7.83463228 6.34707737]
Magic of Scrooge ensemble for N=11: [10.34959861  9.94896495  9.41770607  9.05011501]

Calculating for N=12
Magic of eigenstates for N=12: [10.03499698  9.3883695   8.46085205  6.4354932 ]
Magic of Scrooge ensemble for N=12: [11.34908166 10.9480861  10.41599699 10.0476685 ]

Calculating for N=13
Magic of eigenstates for N=13: [11.07267613 10.45217834  9.55357855  7.09555651]
Magic of Scrooge ensemble for N=13: [12.34873462 11.9476489  11.41546047 11.04712421]

